In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14564,2024-11-21,Lituânia Nkl,13:00,Suduva-Mantinga,Kretinga,1.12,4.97,163.5,1.86,1.80,-12.5,1.95,1.73,vsQ3Y8d2,0.892857,0.201207,0.537634,0.555556,0.094064,108.894,23.109772,0.212223,1.8,1.643168,0.912871,123.22,1.452,0.374927,0.258214,4.0,179.180,65.085977,0.363244,0.0,0.000000,NaN,240.24,2.368,0.901399,0.380658,-63.0,61,66,2.02,3.64,108.542,166.684,0.894043,0.023184,0.084545,-1.00,-0.200,-0.600000,0.794751,0.5,-0.294751,-1.76,-0.352,-11.278409,0.383412,0.6,0.216588
14565,2024-11-21,Europa Eurocopa Feminina,14:00,Klaipedos Neptunas F,Fribourg F,1.47,2.41,132.5,1.83,1.83,-5.5,1.95,1.72,6JOTAoGE,0.680272,0.414938,0.546448,0.546448,0.095210,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,251.640,216.301402,0.859567,1.8,1.643168,0.912871,91.02,4.410,4.326037,0.980961,-24.0,53,82,9.00,1.11,0.000,0.000,0.342619,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
14566,2024-11-21,Lituânia Nkl,13:00,Silute,Delikatesas Joniskis,1.27,3.26,166.5,1.81,1.85,-9.5,2.05,1.65,UVhdvapL,0.787402,0.306748,0.552486,0.540541,0.094150,237.680,114.839116,0.483167,0.6,1.341641,2.236068,207.94,3.028,1.337262,0.441632,-21.0,274.638,109.564387,0.398941,0.0,0.000000,NaN,243.41,3.680,1.698794,0.461629,-97.0,74,101,2.81,2.41,171.476,409.916,0.621255,0.015456,0.152888,-3.48,-0.696,-0.387931,0.571523,0.1,-0.471523,-5.00,-1.000,-2.260000,0.216292,0.3,0.083708
14567,2024-11-21,Lituânia Nkl,13:00,Neptunas 2,Vilkaviskis,1.78,1.89,159.5,1.87,1.79,-2.5,2.05,1.65,YclSYx8r,0.561798,0.529101,0.534759,0.558659,0.090898,126.658,25.803006,0.203722,1.8,1.643168,0.912871,91.76,1.548,0.361483,0.233516,9.0,188.158,70.334240,0.373804,1.2,1.643168,1.369306,125.61,2.322,0.540250,0.232666,19.0,74,79,1.24,1.59,0.000,169.892,0.042388,0.030912,0.152888,0.00,0.000,inf,0.000000,0.0,0.000000,-1.58,-0.316,-2.816456,0.437280,0.5,0.062720
14568,2024-11-21,Lituânia Nkl,13:00,Alytaus Alytus,Rytas 2,1.71,1.97,168.5,1.83,1.83,-2.5,1.97,1.71,M5kltLF8,0.584795,0.507614,0.546448,0.546448,0.092410,173.800,35.603559,0.204854,1.2,1.643168,1.369306,164.64,2.122,0.499119,0.235212,-21.0,238.814,131.080288,0.548880,1.8,1.643168,0.912871,231.29,3.102,2.005984,0.646674,18.0,112,101,1.47,2.29,203.922,394.078,0.099917,0.000000,0.099917,-3.53,-0.706,-1.005666,0.000000,0.0,0.000000,-2.71,-0.542,-1.789668,0.293453,0.3,0.006547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14658,2024-11-21,Eua Ncaa,00:00,Stanford,Norfolk State,1.02,18.00,142.5,1.91,1.91,-21.5,2.00,1.74,Qaag8Bx5,0.980392,0.055556,0.523560,0.523560,0.035948,135.938,67.675112,0.497838,2.4,1.341641,0.559017,95.40,1.806,1.248651,0.691391,70.0,132.104,43.305655,0.327815,1.8,1.643168,0.912871,146.00,1.718,0.496558,0.289033,40.0,90,73,1.06,2.00,97.856,109.644,1.262531,0.000000,0.098314,-0.19,-0.038,-0.526316,0.714604,0.6,-0.114604,-2.33,-0.466,-36.480687,0.570480,0.4,-0.170480
14659,2024-11-21,Eua Ncaa,00:00,Fresno State,Prairie View A&M,1.11,7.50,154.5,1.80,1.95,-14.5,1.95,1.77,vgnNHuLk,0.900901,0.133333,0.555556,0.512821,0.034234,321.172,249.784098,0.777727,1.2,1.6431

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
14568,13:00,Lituânia Nkl,Alytaus Alytus,Rytas 2,1.71,Back Home
